In [67]:
# home : py37Keras

import nltk
import re
import numpy as np
import pandas as pd
import gensim
from nltk.tokenize import sent_tokenize
import multiprocessing
import io
from gensim.models import Phrases
from textblob import TextBlob
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [68]:

text = """This past summer, I had the privilege of participating in the University of Notre Dame’s Research Experience for Undergraduates (REU) program . Under the mentorship of Professor Wendy Bozeman and Professor Georgia Lebedev from the department of Biological Sciences, my goal this summer was to research the effects of cobalt iron oxide cored (CoFe2O3) titanium dioxide (TiO2) nanoparticles as a scaffold for drug delivery, specifically in the delivery of a compound known as curcumin, a flavonoid known for its anti-inflammatory effects. As a high school student trying to find a research opportunity, it was very difficult to find a place that was willing to take me in, but after many months of trying, I sought the help of my high school biology teacher, who used his resources to help me obtain a position in the program.				

Using equipment that a high school student could only dream of using, I was able to map apoptosis (programmed cell death) versus necrosis (cell death due to damage) in HeLa cells, a cervical cancer line, after treating them with curcumin-bound nanoparticles. Using flow cytometry to excite each individually suspended cell with a laser, the scattered light from the cells helped to determine which cells were living, had died from apoptosis or had died from necrosis. Using this collected data, it was possible to determine if the curcumin and/or the nanoparticles had played any significant role on the cervical cancer cells. Later, I was able to image cells in 4D through con-focal microscopy. From growing HeLa cells to trying to kill them with different compounds, I was able to gain the hands-on experience necessary for me to realize once again why I love science.				

Living on the Notre Dame campus with other REU students, UND athletes, and other summer school students was a whole other experience that prepared me for the world beyond high school. For 9 weeks, I worked, played and bonded with the other students, and had the opportunity to live the life of an independent college student.				

Along with the individually tailored research projects and the housing opportunity, there were seminars on public speaking, trips to the Fermi National Accelerator Laboratory, and one-on-one writing seminars for the end of the summer research papers we were each required to write. By the end of the summer, I wasn’t ready to leave the research that I was doing. While my research didn’t yield definitive results for the effects of curcumin on cervical cancer cells, my research on curcumin-functionalized CoFe2O4/TiO2 core-shell nanoconjugates indicated that there were many unknown factors affecting the HeLa cells, and spurred the lab to expand their research into determining whether or not the timing of the drug delivery mattered and whether or not the position of the binding site of the drugs would alter the results. Through this summer experience, I realized my ambition to pursue a career in research. I always knew that I would want to pursue a future in science, but the exciting world of research where the discoveries are limitless has captured my heart. This school year, the REU program has offered me a year-long job, and despite my obligations as a high school senior preparing for college, I couldn’t give up this offer, and so during this school year, I will be able to further both my research and interest in nanotechnology. """


In [69]:
essay_input_corpus = str(text) #문장입력
essay_input_corpus = essay_input_corpus.lower()#소문자 변환
print('essay_input_corpus :', essay_input_corpus)

sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화 > 문장으로 구분
total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
total_words = len(word_tokenize(essay_input_corpus))# 총 단어수
print(total_words)
split_sentences = []
for sentence in sentences:
    processed = re.sub("[^a-zA-Z]"," ", sentence)
    words = processed.split()
    split_sentences.append(words)

essay_input_corpus : this past summer, i had the privilege of participating in the university of notre dame’s research experience for undergraduates (reu) program . under the mentorship of professor wendy bozeman and professor georgia lebedev from the department of biological sciences, my goal this summer was to research the effects of cobalt iron oxide cored (cofe2o3) titanium dioxide (tio2) nanoparticles as a scaffold for drug delivery, specifically in the delivery of a compound known as curcumin, a flavonoid known for its anti-inflammatory effects. as a high school student trying to find a research opportunity, it was very difficult to find a place that was willing to take me in, but after many months of trying, i sought the help of my high school biology teacher, who used his resources to help me obtain a position in the program.				

using equipment that a high school student could only dream of using, i was able to map apoptosis (programmed cell death) versus necrosis (cell death

In [70]:
split_sentences

[['this',
  'past',
  'summer',
  'i',
  'had',
  'the',
  'privilege',
  'of',
  'participating',
  'in',
  'the',
  'university',
  'of',
  'notre',
  'dame',
  's',
  'research',
  'experience',
  'for',
  'undergraduates',
  'reu',
  'program'],
 ['under',
  'the',
  'mentorship',
  'of',
  'professor',
  'wendy',
  'bozeman',
  'and',
  'professor',
  'georgia',
  'lebedev',
  'from',
  'the',
  'department',
  'of',
  'biological',
  'sciences',
  'my',
  'goal',
  'this',
  'summer',
  'was',
  'to',
  'research',
  'the',
  'effects',
  'of',
  'cobalt',
  'iron',
  'oxide',
  'cored',
  'cofe',
  'o',
  'titanium',
  'dioxide',
  'tio',
  'nanoparticles',
  'as',
  'a',
  'scaffold',
  'for',
  'drug',
  'delivery',
  'specifically',
  'in',
  'the',
  'delivery',
  'of',
  'a',
  'compound',
  'known',
  'as',
  'curcumin',
  'a',
  'flavonoid',
  'known',
  'for',
  'its',
  'anti',
  'inflammatory',
  'effects'],
 ['as',
  'a',
  'high',
  'school',
  'student',
  'trying',

In [71]:
lemmatizer = WordNetLemmatizer()
preprossed_sent_all = []
for i in split_sentences:
    preprossed_sent = []
    for i_ in i:
        if i_ not in stop: #remove stopword
            lema_re = lemmatizer.lemmatize(i_, pos='v') #표제어 추출, 동사는 현재형으로 변환, 3인칭 단수는 1인칭으로 변환
            if len(lema_re) > 3: # 단어 길이가 3 초과단어만 저장(길이가 3 이하는 제거)
                preprossed_sent.append(lema_re)
    preprossed_sent_all.append(preprossed_sent)
preprossed_sent_all

[['past',
  'summer',
  'privilege',
  'participate',
  'university',
  'notre',
  'dame',
  'research',
  'experience',
  'undergraduates',
  'program'],
 ['mentorship',
  'professor',
  'wendy',
  'bozeman',
  'professor',
  'georgia',
  'lebedev',
  'department',
  'biological',
  'sciences',
  'goal',
  'summer',
  'research',
  'effect',
  'cobalt',
  'iron',
  'oxide',
  'core',
  'cofe',
  'titanium',
  'dioxide',
  'nanoparticles',
  'scaffold',
  'drug',
  'delivery',
  'specifically',
  'delivery',
  'compound',
  'know',
  'curcumin',
  'flavonoid',
  'know',
  'anti',
  'inflammatory',
  'effect'],
 ['high',
  'school',
  'student',
  'find',
  'research',
  'opportunity',
  'difficult',
  'find',
  'place',
  'will',
  'take',
  'many',
  'months',
  'seek',
  'help',
  'high',
  'school',
  'biology',
  'teacher',
  'resources',
  'help',
  'obtain',
  'position',
  'program'],
 ['equipment',
  'high',
  'school',
  'student',
  'could',
  'dream',
  'able',
  'apoptosis'

In [72]:
# 역토큰화
detokenized = []
for i_token in range(len(preprossed_sent_all)):
    for tk in preprossed_sent_all:
        t = ' '.join(tk)
        detokenized.append(t)
        
detokenized

['past summer privilege participate university notre dame research experience undergraduates program',
 'mentorship professor wendy bozeman professor georgia lebedev department biological sciences goal summer research effect cobalt iron oxide core cofe titanium dioxide nanoparticles scaffold drug delivery specifically delivery compound know curcumin flavonoid know anti inflammatory effect',
 'high school student find research opportunity difficult find place will take many months seek help high school biology teacher resources help obtain position program',
 'equipment high school student could dream able apoptosis program cell death versus necrosis cell death damage hela cells cervical cancer line treat curcumin bind nanoparticles',
 'flow cytometry excite individually suspend cell laser scatter light cells help determine cells live apoptosis necrosis',
 'collect data possible determine curcumin nanoparticles play significant role cervical cancer cells',
 'later able image cells focal

In [73]:
#  TF-IDF 행렬로 변환
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=None)
X = vectorizer.fit_transform(detokenized)
X.shape

(256, 158)

In [74]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', random_state=777, max_iter=1)
lda_top = lda_model.fit_transform(X)
print(lda_model.components_)
print(lda_model.components_.shape)

[[0.84539644 0.64764981 0.76375597 ... 0.70702709 0.8883679  0.5990078 ]
 [0.67610203 0.61003747 0.89463223 ... 0.94870593 0.75132187 0.831808  ]
 [1.57798013 0.90347048 0.6343751  ... 0.9343548  2.73403219 0.63813197]
 ...
 [0.60415736 1.68442274 0.6829438  ... 2.41076506 0.71151374 0.73682249]
 [0.67416327 0.83092446 0.89459121 ... 0.82267755 0.59919564 0.82970969]
 [1.02125758 0.87933184 0.92075364 ... 0.78576855 0.96954506 0.68516586]]
(10, 158)


In [98]:
terms = vectorizer.get_feature_names() 
# 단어 집합. 1,000개의 단어가 저장되어있음.
topics_ext = []
def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d :" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n -1:-1]])
        topics_ext.append([(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n -1:-1]])
        


In [99]:
topics_ext.append(get_topics(lda_model.components_, terms))

Topic 1 : [('collect', 1.61), ('significant', 1.53), ('data', 1.52), ('cancer', 1.51), ('possible', 1.5)]
Topic 2 : [('significant', 1.6), ('possible', 1.51), ('data', 1.51), ('role', 1.51), ('collect', 1.51)]
Topic 3 : [('school', 3.61), ('year', 2.73), ('dame', 2.69), ('notre', 2.63), ('experience', 2.43)]
Topic 4 : [('ambition', 2.77), ('career', 2.69), ('realize', 2.38), ('pursue', 2.34), ('experience', 2.2)]
Topic 5 : [('able', 2.6), ('microscopy', 2.6), ('focal', 2.48), ('image', 2.46), ('cells', 2.45)]
Topic 6 : [('result', 2.14), ('curcumin', 2.1), ('death', 2.04), ('cells', 2.04), ('research', 1.92)]
Topic 7 : [('ready', 2.54), ('leave', 2.47), ('know', 2.42), ('summer', 2.13), ('research', 2.1)]
Topic 8 : [('seminars', 2.48), ('write', 2.41), ('opportunity', 2.27), ('work', 2.16), ('life', 2.15)]
Topic 9 : [('leave', 1.61), ('ready', 1.52), ('research', 1.41), ('summer', 1.28), ('students', 1.15)]
Topic 10 : [('help', 2.91), ('high', 2.21), ('school', 2.19), ('place', 1.91), 

In [101]:
topics_ext

[[('collect', 1.61),
  ('significant', 1.53),
  ('data', 1.52),
  ('cancer', 1.51),
  ('possible', 1.5)],
 [('significant', 1.6),
  ('possible', 1.51),
  ('data', 1.51),
  ('role', 1.51),
  ('collect', 1.51)],
 [('school', 3.61),
  ('year', 2.73),
  ('dame', 2.69),
  ('notre', 2.63),
  ('experience', 2.43)],
 [('ambition', 2.77),
  ('career', 2.69),
  ('realize', 2.38),
  ('pursue', 2.34),
  ('experience', 2.2)],
 [('able', 2.6),
  ('microscopy', 2.6),
  ('focal', 2.48),
  ('image', 2.46),
  ('cells', 2.45)],
 [('result', 2.14),
  ('curcumin', 2.1),
  ('death', 2.04),
  ('cells', 2.04),
  ('research', 1.92)],
 [('ready', 2.54),
  ('leave', 2.47),
  ('know', 2.42),
  ('summer', 2.13),
  ('research', 2.1)],
 [('seminars', 2.48),
  ('write', 2.41),
  ('opportunity', 2.27),
  ('work', 2.16),
  ('life', 2.15)],
 [('leave', 1.61),
  ('ready', 1.52),
  ('research', 1.41),
  ('summer', 1.28),
  ('students', 1.15)],
 [('help', 2.91),
  ('high', 2.21),
  ('school', 2.19),
  ('place', 1.91),
  ('

In [141]:
topics_ext = list(filter(None, topics_ext))

In [142]:
topics_ext[0][0][0]

'collect'

In [143]:
topics_ext[0][1][0]

'significant'

In [144]:
topics_ext[0][2][0]

'data'

In [145]:
topics_ext[1][2][0]

'data'

In [146]:
len(topics_ext)

10

In [147]:
result_ = []
cnt = 0
cnt_ = 0
for ittm in range(len(topics_ext)):
    print('ittm:', ittm)
    cnt_ = 0
    for t in range(len(topics_ext[ittm])-1):
        
        print('t:', t)
        add = topics_ext[ittm][cnt_][0]
        result_.append(add)
        print('result_:', result_)
        cnt_ += 1
result_

ittm: 0
t: 0
result_: ['collect']
t: 1
result_: ['collect', 'significant']
t: 2
result_: ['collect', 'significant', 'data']
t: 3
result_: ['collect', 'significant', 'data', 'cancer']
ittm: 1
t: 0
result_: ['collect', 'significant', 'data', 'cancer', 'significant']
t: 1
result_: ['collect', 'significant', 'data', 'cancer', 'significant', 'possible']
t: 2
result_: ['collect', 'significant', 'data', 'cancer', 'significant', 'possible', 'data']
t: 3
result_: ['collect', 'significant', 'data', 'cancer', 'significant', 'possible', 'data', 'role']
ittm: 2
t: 0
result_: ['collect', 'significant', 'data', 'cancer', 'significant', 'possible', 'data', 'role', 'school']
t: 1
result_: ['collect', 'significant', 'data', 'cancer', 'significant', 'possible', 'data', 'role', 'school', 'year']
t: 2
result_: ['collect', 'significant', 'data', 'cancer', 'significant', 'possible', 'data', 'role', 'school', 'year', 'dame']
t: 3
result_: ['collect', 'significant', 'data', 'cancer', 'significant', 'possible',

['collect',
 'significant',
 'data',
 'cancer',
 'significant',
 'possible',
 'data',
 'role',
 'school',
 'year',
 'dame',
 'notre',
 'ambition',
 'career',
 'realize',
 'pursue',
 'able',
 'microscopy',
 'focal',
 'image',
 'result',
 'curcumin',
 'death',
 'cells',
 'ready',
 'leave',
 'know',
 'summer',
 'seminars',
 'write',
 'opportunity',
 'work',
 'leave',
 'ready',
 'research',
 'summer',
 'help',
 'high',
 'school',
 'place']

In [150]:
result_fin = list(set(result_))

In [151]:
# 입력문장에서 추출한 최종 키워드들(중복값을 제거한 값)
result_fin

['research',
 'notre',
 'seminars',
 'write',
 'ready',
 'curcumin',
 'collect',
 'cancer',
 'career',
 'focal',
 'cells',
 'image',
 'year',
 'ambition',
 'know',
 'work',
 'significant',
 'data',
 'result',
 'possible',
 'school',
 'summer',
 'high',
 'help',
 'place',
 'death',
 'able',
 'realize',
 'pursue',
 'opportunity',
 'leave',
 'dame',
 'microscopy',
 'role']